## Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural similar a do Bengio 2003 para prever a próxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Linguagem".

Portanto, você deve implementar o modelo de linguagem inspirado no artigo do Bengio, para prever a próxima palavra usando rede com embeddings e duas camadas.
Sugestão de alguns parâmetros:
* context_size = 9
* max_vocab_size = 3000
* embedding_dim = 64
* usar pontuação no vocabulário
* descartar qualquer contexto ou target que não esteja no vocabulário
* É esperado conseguir uma perplexidade da ordem de 50.
* Procurem fazer asserts para garantir que partes do seu programa estão testadas

Este enunciado não é fixo, podem mudar qualquer um dos parâmetros acima, mas procurem conseguir a perplexidade esperada ou menor.

Gerem alguns frases usando um contexto inicial e depois deslocando o contexto e prevendo a próxima palavra gerando frases compridas para ver se está gerando texto plausível.

Algumas dicas:
- Inclua caracteres de pontuação (ex: `.` e `,`) no vocabulário.
- Deixe tudo como caixa baixa (lower-case).
- A escolha do tamanho do vocabulario é importante: ser for muito grande, fica difícil para o modelo aprender boas representações. Se for muito pequeno, o modelo apenas conseguirá gerar textos simples.
- Remova qualquer exemplo de treino/validação/teste que tenha pelo menos um token desconhecido (ou na entrada ou na saída).
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

Procure por `TODO` para entender onde você precisa inserir o seu código.

## Faz download e carrega o dataset

In [29]:
# define the environment (local/colab)
environment = 'local'

In [27]:
# local
# !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8 -P data/
# !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8 -P data/

# colab
# !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
# !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8


In [30]:

if environment == 'local':
    file1 = 'data/67724.txt.utf-8'
    file2 = 'data/67725.txt.utf-8'
else:
    file1 = '67724.txt.utf-8'
    file2 = '67725.txt.utf-8'

text = open(file1,"r").read() +  open(file2,"r").read()

paragraphs = text.split("\n\n")
len(paragraphs)

4969

In [3]:
# remove all spaces and convert to lower case, dont remove . and , as they are important for tokenization
from unidecode import unidecode

cleaned_paragraphs = [unidecode(paragraph.replace("\n", " ").lower()) for paragraph in paragraphs if paragraph.strip()]

len(cleaned_paragraphs)

4892

## Análise do dataset

In [4]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(r'\w+', text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12395

## Criando um vocabulário

In [5]:
vocab_size = 3000
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [6]:
len(vocab)

3000

In [7]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(r'\w+', sentence.lower())]

encode_sentence(cleaned_paragraphs[20], vocab)

[0,
 139,
 0,
 19,
 0,
 6,
 44,
 109,
 243,
 262,
 2666,
 10,
 1061,
 0,
 2,
 184,
 131,
 282,
 4,
 2256,
 6,
 0,
 1,
 2669]

## Classe do dataset

In [8]:
context_size = 5 # 5 palavras de entrada. O target é a próxima palavra
max_vocab_size = 3000
embedding_dim = 64
debug = 100

In [9]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

In [10]:
"""TODO: implemente a classe do dataset"""
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class MyDataset(Dataset):
    def __init__(self, data, vocab, context_size):
        self.data = data
        self.vocab = vocab
        self.context_size = context_size
        self.x = []
        self.y = []
        for sentence in data:
            words = encode_sentence(sentence, vocab)
            for i in range(len(words) - context_size):
                self.x.append(words[i:i+context_size])
                self.y.append(words[i+context_size])
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])

# if debug limit data to 10 samples
if debug:
    train = train[:debug]
    val = val[:debug]

train_data = MyDataset(train, vocab, context_size)
val_data = MyDataset(val, vocab, context_size)

In [11]:
batch_size = 2
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
sample = next(iter(train_loader))

## Model

In [12]:
import torch.nn as nn

# Bengio 2023
class LanguageModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(LanguageModel, self).__init__()
        
        # Look up table
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Linear layer
        self.linear1 = nn.Linear(context_size*embedding_dim, 128)

        # Linear layer
        self.linear2 = nn.Linear(128, vocab_size)
    def forward(self, x):
        x = self.embedding(x).view(-1, context_size*embedding_dim)
        x = torch.tanh(self.linear1(x))
        x = self.linear2(x)
        return x

In [13]:
model = LanguageModel(vocab_size, embedding_dim, context_size)

In [14]:
# sample = next(iter(train_loader))
_input = sample[0]
target = sample[1]

In [15]:
output = model(_input)

In [16]:
output.argmax(dim=1)[0].item()

1133

In [17]:
vocab

{'a': 1,
 'que': 2,
 'e': 3,
 'o': 4,
 'de': 5,
 'se': 6,
 'um': 7,
 'do': 8,
 'nao': 9,
 'uma': 10,
 'da': 11,
 'os': 12,
 'com': 13,
 'sua': 14,
 'para': 15,
 'as': 16,
 'seu': 17,
 'pery': 18,
 'em': 19,
 'no': 20,
 'por': 21,
 'ao': 22,
 'como': 23,
 'lhe': 24,
 'd': 25,
 'tinha': 26,
 'era': 27,
 'cecilia': 28,
 'na': 29,
 'sobre': 30,
 'mas': 31,
 'elle': 32,
 'the': 33,
 'dos': 34,
 'indio': 35,
 'me': 36,
 'seus': 37,
 'mais': 38,
 'vos': 39,
 'antonio': 40,
 'quando': 41,
 'alvaro': 42,
 'das': 43,
 'disse': 44,
 'of': 45,
 'nos': 46,
 'ella': 47,
 'te': 48,
 'olhos': 49,
 'senhora': 50,
 'menina': 51,
 'pela': 52,
 'tu': 53,
 'depois': 54,
 'isabel': 55,
 'havia': 56,
 'gutenberg': 57,
 'fidalgo': 58,
 'casa': 59,
 'estava': 60,
 'ainda': 61,
 'tempo': 62,
 'ja': 63,
 'mariz': 64,
 'project': 65,
 'aventureiros': 66,
 'so': 67,
 'momento': 68,
 'loredano': 69,
 'mesmo': 70,
 'italiano': 71,
 'todos': 72,
 'pelo': 73,
 'vida': 74,
 'sem': 75,
 'dous': 76,
 'to': 77,
 'homem': 

In [18]:
# convert to tokens
def convert_to_tokens(indexes, vocab):
    return [list(vocab.keys())[i-1] for i in indexes]

print(convert_to_tokens(_input[0], vocab))
print(convert_to_tokens([target[0].item()], vocab))

convert_to_tokens([output.argmax(dim=1)[0].item()], vocab)

['com', 'as', 'suas', 'azas', 'a']
['alma']


['puro']

## Training

In [19]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [20]:
epochs = 10
lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
model.to(device)

LanguageModel(
  (embedding): Embedding(3000, 64)
  (linear1): Linear(in_features=320, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=3000, bias=True)
)

In [21]:
# """TODO: Implemente o loop de treinamento. Em cada época, calcule e imprima a loss no dataset de validação"""

for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"Epoch {epoch+1}/{epochs}")
    for i, (_input, target) in enumerate(train_loader):
        _input, target = _input.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(_input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    perplexity = torch.exp(torch.tensor(total_loss/len(train_loader)))
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}, Perplexity: {perplexity.item()}")

    model.eval()
    total_loss = 0
    with torch.no_grad():
        for i, (_input, target) in enumerate(val_loader):
            _input, target = _input.to(device), target.to(device)
            output = model(_input)
            loss = criterion(output, target)
            total_loss += loss.item()

    perplexity = torch.exp(torch.tensor(total_loss/len(val_loader)))
    print(f"Validation Loss: {total_loss/len(val_loader)}, Perplexity: {perplexity.item()}\n")

Epoch 1/10
Epoch 1/10, Loss: 6.5444397914422865, Perplexity: 695.3670043945312
Validation Loss: 6.4309457196427955, Perplexity: 620.7608032226562

Epoch 2/10
Epoch 2/10, Loss: 4.346570234981414, Perplexity: 77.21316528320312
Validation Loss: 6.646197207020395, Perplexity: 769.8512573242188

Epoch 3/10
Epoch 3/10, Loss: 3.0181282429266454, Perplexity: 20.452970504760742
Validation Loss: 6.916073725191818, Perplexity: 1008.3532104492188

Epoch 4/10
Epoch 4/10, Loss: 1.8309487695996955, Perplexity: 6.239803791046143
Validation Loss: 7.212626721988315, Perplexity: 1356.450927734375

Epoch 5/10
Epoch 5/10, Loss: 0.957812468323869, Perplexity: 2.605989694595337
Validation Loss: 7.641770141578624, Perplexity: 2083.428955078125

Epoch 6/10
Epoch 6/10, Loss: 0.4742612517071882, Perplexity: 1.6068267822265625
Validation Loss: 7.74283735419003, Perplexity: 2305.00341796875

Epoch 7/10
Epoch 7/10, Loss: 0.22888676469295918, Perplexity: 1.2571996450424194
Validation Loss: 8.022131172972545, Perplex

## Avaliação

In [25]:
""" TODO: calcule a perplexidade final no dataset de validação """

# Perplexidade final

model.eval()
total_loss = 0
with torch.no_grad():
    for i, (_input, target) in enumerate(val_loader):
        _input, target = _input.to(device), target.to(device)
        output = model(_input)
        loss = criterion(output, target)
        total_loss += loss.item()
    
perplexity = torch.exp(torch.tensor(total_loss/len(val_loader)))
print(f"Final Perplexity: {perplexity.item()}")

Final Perplexity: 4980.0537109375


## Exemplo de uso

In [26]:
text = "um dia a praia irá"

def generate_text(model, vocab, text, max_length):
    """TODO: implemente a função para gerar texto até atingir o max_length"""
    model.eval()
    words = text.split(" ")
    for i in range(max_length):
        input_ids = encode_sentence(" ".join(text.split()[-context_size:]), vocab)
        _input = torch.tensor([input_ids]).to(device)
        output = model(_input)
        word = convert_to_tokens([output.argmax(dim=1).item()], vocab)[0]
        words.append(word)
    return " ".join(words)

context = 5
max_length= 15
generate_text(model, vocab, text, max_length)

'um dia a praia irá de de de de de de de de de de de de de de de'